In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
import google.genai as genai
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
loader = PyPDFLoader("data/textbook.pdf")
documents = loader.load()


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
texts = text_splitter.split_documents(documents)

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004"
)

In [ ]:
vectorstore = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory="chroma_db"
)


In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)


In [ ]:
prompt = ChatPromptTemplate.from_template(
    """
    You are a helpful study assistant.
    Use ONLY the context below to answer the question.

    Context:
    {context}

    Question:
    {question}

    Answer:
    """
)


In [ ]:
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",  # Correct format with 'models/' prefix
    temperature=0.2
)

In [ ]:
rag_chain = (
    {"context": retriever,
     "question": RunnablePassthrough()
    }
    | prompt | llm
)

In [ ]:
# First, let's check what models are available
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# List available models
print("Available models:")
for model in genai.list_models():
    if 'generateContent' in model.supported_generation_methods:
        print(f"- {model.name}")

In [ ]:
query = "What is the general idea of The Evolution of Natural Philosophy into Modern Physics"
response = rag_chain.invoke(query)

print(response.content)
